In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Deep Convolutional GAN (DCGAN) Implementation with MNIST using PyTorch

This guide will walk you through the implementation of a Deep Convolutional GAN (DCGAN) using PyTorch. DCGANs are a class of generative adversarial networks (GANs) specifically designed for generating high-quality images.


# Some Theory

## Generative Adversial Networks (GANs)

Generative Adversarial Networks (GANs) are a class of deep learning models introduced by Ian Goodfellow and his colleagues in 2014. GANs consist of two neural networks, the Generator and the Discriminator, trained simultaneously through a competitive process.

- Generator: The Generator takes random noice as input and generates synthetic data samples, such as images
- Discriminator: Binary Classifier that distinguishes between real data and fake data generated by Generator

# Deep Convolutional GANs (DCGANs)

Deep Convolutional GANs (DCGANs) are a variant of GANs proposed by Alec Radford, Luke Metz, and Soumith Chintala in 2015. DCGANs introduce architectural guidelines and best practices for designing both the Generator and Discriminator networks, specifically tailored for generating high-quality images.

### Key characteristics of DCGANs include:

- Convolutional Networks: Both the Generator and Discriminator are based on convolutional neural networks (CNNs), which are well-suited for image processing tasks.

- Strided Convolutions and Fractional Strided Convolutions: DCGANs use strided convolutions in the Generator to upsample the input noise, and fractional strided (transpose) convolutions in the Discriminator to downsample the image.

- Batch Normalization: Batch normalization is applied in both the Generator and Discriminator to stabilize and accelerate training.

- No Fully Connected Layers: DCGANs avoid the use of fully connected layers in favor of global average pooling in the Discriminator and fully convolutional architectures in the Generator.

In [2]:
import torch
import torch.nn as nn

if torch.cuda.is_available():
    print(torch.cuda.get_device_name())

In [3]:
torch.cuda.is_available()

False

# Generator

The Generator consists of several layers of transposed convolutions (also known as fractionally-strided convolutions) interleaved with batch normalization and ReLU activation functions. It gradually upsamples the input noise vector into a synthetic image.

- nz - size of the input noice vector (latent vector)
- ngf - number of features. This parameter controls the complexity of the network
- nc - Number of channels in the output image (3 for RGB images), 1 for MNIST dataset

In [4]:
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # First convolutional layer
            nn.ConvTranspose2d(nz, ngf*4, 7, 1, 0, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            # Second convolutional layer
            nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf*2),
            nn.ReLU(True),
            # Third convolutional layer
            nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # Output layer
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        
    def forward(self, input):
        return self.main(input)
    
nz = 100
ngf = 64
nc = 1

generator = Generator(nz,ngf,nc)

In [5]:
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # Input convolutional layer
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # Second convolutional layer
            nn.Conv2d(ndf, ndf*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*2),
            nn.LeakyReLU(0.2, inplace=True),
            # Third convolutional layer
            nn.Conv2d(ndf*2, ndf*4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf*4),
            nn.LeakyReLU(0.2, inplace=True),
            # Output layer
            nn.Conv2d(ndf*4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

ndf = 64 
nc = 1 

discriminator = Discriminator(nc, ndf)
